In [ ]:
%pip install -U opensearch-py==2.3.1
%pip install -U retrying==1.3.4

In [1]:
import boto3
import time
import random
import pprint as pp
import uuid
import json
from retrying import retry
from utility import create_bedrock_execution_role, create_oss_policy_attach_bedrock_execution_role, create_policies_in_oss
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth

# auth for opensearch
boto3_session = boto3.Session()
region_name = boto3_session.region_name
sts_client = boto3.client('sts')
account_id = sts_client.get_caller_identity()["Account"]
credentials = boto3_session.get_credentials()

# opensearch service
service = 'aoss'
awsauth = auth = AWSV4SignerAuth(credentials, region_name, service)


bedrock_agent_client = boto3_session.client('bedrock-agent', region_name=region_name)

# bucket and parameter stored from lab01
%store -r bucket
%store -r prefix
%store -r yml_dir
%store -r uml_dir

## Create a vector store - OpenSearch Serverless index

### Step 1 - Create OSS policies and collection
Firt of all we have to create a vector store. In this section we will use *Amazon OpenSerach serverless.*

Amazon OpenSearch Serverless is a serverless option in Amazon OpenSearch Service. As a developer, you can use OpenSearch Serverless to run petabyte-scale workloads without configuring, managing, and scaling OpenSearch clusters. You get the same interactive millisecond response times as OpenSearch Service with the simplicity of a serverless environment. Pay only for what you use by automatically scaling resources to provide the right amount of capacity for your application—without impacting data ingestion.

In [2]:
suffix = random.randrange(200, 900)
vector_store_name = f'swagger-api-{suffix}'
index_name = f"swagger-api-{suffix}"
aoss_client = boto3_session.client('opensearchserverless')
bedrock_kb_execution_role = create_bedrock_execution_role(bucket_name=bucket)
bedrock_kb_execution_role_arn = bedrock_kb_execution_role['Role']['Arn']

In [3]:
# create security, network and data access policies within OSS
encryption_policy, network_policy, access_policy = create_policies_in_oss(vector_store_name=vector_store_name,
                       aoss_client=aoss_client,
                       bedrock_kb_execution_role_arn=bedrock_kb_execution_role_arn)
collection = aoss_client.create_collection(name=vector_store_name,type='VECTORSEARCH')

In [4]:
pp.pprint(collection)
time.sleep(10)

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '307',
                                      'content-type': 'application/x-amz-json-1.0',
                                      'date': 'Wed, 22 May 2024 01:22:46 GMT',
                                      'x-amzn-requestid': 'd3d53908-a8c3-4b86-8724-162ac44c89fc'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'd3d53908-a8c3-4b86-8724-162ac44c89fc',
                      'RetryAttempts': 0},
 'createCollectionDetail': {'arn': 'arn:aws:aoss:us-west-2:376678947624:collection/fonhxxn4l0bklnc3cw76',
                            'createdDate': 1716340966495,
                            'id': 'fonhxxn4l0bklnc3cw76',
                            'kmsKeyArn': 'auto',
                            'lastModifiedDate': 1716340966495,
                            'name': 'swagger-api-551',
                            'standbyReplicas': 'ENABLED',

In [5]:
collection_id = collection['createCollectionDetail']['id']
host = collection_id + '.' + region_name + '.aoss.amazonaws.com'
print(host)

fonhxxn4l0bklnc3cw76.us-west-2.aoss.amazonaws.com


In [6]:
# create oss policy and attach it to Bedrock execution role
create_oss_policy_attach_bedrock_execution_role(collection_id=collection_id,
                                                bedrock_kb_execution_role=bedrock_kb_execution_role)

Opensearch serverless arn:  arn:aws:iam::376678947624:policy/AmazonBedrockOSSPolicyForKnowledgeBase_556


### Step 2 - Create vector index

In [7]:
index_name = f"bedrock-sample-index-{suffix}"
body_json = {
   "settings": {
      "index.knn": "true"
   },
   "mappings": {
      "properties": {
         "vector": {
            "type": "knn_vector",
            "dimension": 1536,
            "method": {
                "name": "hnsw",
                "space_type": "innerproduct",
                "engine": "faiss",
                "parameters": {
                  "ef_construction": 256,
                  "m": 48
                }
             }
         },
         "text": {
            "type": "text"
         },
         "text-metadata": {
            "type": "text"         
         }
      }
   }
}
# Build the OpenSearch client
oss_client = OpenSearch(
    hosts=[{'host': host, 'port': 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300
)
# # It can take up to a minute for data access rules to be enforced
time.sleep(60)

In [8]:
# Create index
response = oss_client.indices.create(index=index_name, body=json.dumps(body_json))
print('\nCreating index:')
print(response)


Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'bedrock-sample-index-551'}


## Create Knowledge Base
Steps:
- initialize Open search serverless configuration which will include collection ARN, index name, vector field, text field and metadata field.
- initialize chunking strategy, based on which KB will split the documents into pieces of size equal to the chunk size mentioned in the `chunkingStrategyConfiguration`.
- initialize the s3 configuration, which will be used to create the data source object later.
- initialize the Titan embeddings model ARN, as this will be used to create the embeddings for each of the text chunks.

In [9]:
opensearchServerlessConfiguration = {
            "collectionArn": collection["createCollectionDetail"]['arn'],
            "vectorIndexName": index_name,
            "fieldMapping": {
                "vectorField": "vector",
                "textField": "text",
                "metadataField": "text-metadata"
            }
        }

chunkingStrategyConfiguration = {
    "chunkingStrategy": "NONE",
}

s3Configuration = {
    "bucketArn": f"arn:aws:s3:::{bucket}",
    "inclusionPrefixes":[f"{prefix}/yml_questions/"] # you can use this if you want to create a KB using data within s3 prefixes.
}

embeddingModelArn = f"arn:aws:bedrock:{region_name}::foundation-model/amazon.titan-embed-text-v1"

kb_name = f"bedrock-sample-knowledge-base-{suffix}"
description = "Swagger OpenAPI knowledge base."
roleArn = bedrock_kb_execution_role_arn

Provide the above configurations as input to the `create_knowledge_base` method, which will create the Knowledge base.

In [10]:
# Create a KnowledgeBase
from retrying import retry

@retry(wait_random_min=1000, wait_random_max=2000,stop_max_attempt_number=7)
def create_knowledge_base_func():
    create_kb_response = bedrock_agent_client.create_knowledge_base(
        name = kb_name,
        description = description,
        roleArn = roleArn,
        knowledgeBaseConfiguration = {
            "type": "VECTOR",
            "vectorKnowledgeBaseConfiguration": {
                "embeddingModelArn": embeddingModelArn
            }
        },
        storageConfiguration = {
            "type": "OPENSEARCH_SERVERLESS",
            "opensearchServerlessConfiguration":opensearchServerlessConfiguration
        }
    )
    return create_kb_response["knowledgeBase"]

In [11]:
try:
    kb = create_knowledge_base_func()
except Exception as err:
    print(f"{err=}, {type(err)=}")

Next we need to create a data source, which will be associated with the knowledge base created above. Once the data source is ready, we can then start to ingest the documents.

In [12]:
# Get KnowledgeBase 
get_kb_response = bedrock_agent_client.get_knowledge_base(knowledgeBaseId = kb['knowledgeBaseId'])

In [13]:
# Create a DataSource in KnowledgeBase 
create_ds_response = bedrock_agent_client.create_data_source(
    name = kb_name,
    description = description,
    knowledgeBaseId = kb['knowledgeBaseId'],
    dataSourceConfiguration = {
        "type": "S3",
        "s3Configuration":s3Configuration
    },
    vectorIngestionConfiguration = {
        "chunkingConfiguration": chunkingStrategyConfiguration
    }
)
ds = create_ds_response["dataSource"]
# # It can take up to a minute for data access rules to be enforced
time.sleep(20)
pp.pprint(ds)

{'createdAt': datetime.datetime(2024, 5, 22, 1, 24, 3, 873323, tzinfo=tzlocal()),
 'dataDeletionPolicy': 'DELETE',
 'dataSourceConfiguration': {'s3Configuration': {'bucketArn': 'arn:aws:s3:::sagemaker-us-west-2-376678947624',
                                                 'inclusionPrefixes': ['swagger_codegen/yml_questions/']},
                             'type': 'S3'},
 'dataSourceId': 'EQYVUCB3BM',
 'description': 'Swagger OpenAPI knowledge base.',
 'knowledgeBaseId': '4NQZSAPNPH',
 'name': 'bedrock-sample-knowledge-base-551',
 'status': 'AVAILABLE',
 'updatedAt': datetime.datetime(2024, 5, 22, 1, 24, 3, 873323, tzinfo=tzlocal()),
 'vectorIngestionConfiguration': {'chunkingConfiguration': {'chunkingStrategy': 'NONE'}}}


### Start ingestion job
Once the KB and data source is created, we can start the ingestion job.
During the ingestion job, KB will fetch the documents in the data source, pre-process it to extract text, chunk it based on the chunking size provided, create embeddings of each chunk and then write it to the vector database, in this case OSS.

In [14]:
# Start an ingestion job
start_job_response = bedrock_agent_client.start_ingestion_job(knowledgeBaseId = kb['knowledgeBaseId'], dataSourceId = ds["dataSourceId"])

In [15]:
job = start_job_response["ingestionJob"]
pp.pprint(job)

{'dataSourceId': 'EQYVUCB3BM',
 'ingestionJobId': 'VPHOIVZB0H',
 'knowledgeBaseId': '4NQZSAPNPH',
 'startedAt': datetime.datetime(2024, 5, 22, 1, 24, 24, 636367, tzinfo=tzlocal()),
 'statistics': {'numberOfDocumentsDeleted': 0,
                'numberOfDocumentsFailed': 0,
                'numberOfDocumentsScanned': 0,
                'numberOfMetadataDocumentsModified': 0,
                'numberOfMetadataDocumentsScanned': 0,
                'numberOfModifiedDocumentsIndexed': 0,
                'numberOfNewDocumentsIndexed': 0},
 'status': 'STARTING',
 'updatedAt': datetime.datetime(2024, 5, 22, 1, 24, 24, 636367, tzinfo=tzlocal())}


In [16]:
# Get job 
while(job['status']!='COMPLETE' ):
  get_job_response = bedrock_agent_client.get_ingestion_job(
      knowledgeBaseId = kb['knowledgeBaseId'],
        dataSourceId = ds["dataSourceId"],
        ingestionJobId = job["ingestionJobId"]
  )
  job = get_job_response["ingestionJob"]
pp.pprint(job)
time.sleep(80)

{'dataSourceId': 'EQYVUCB3BM',
 'ingestionJobId': 'VPHOIVZB0H',
 'knowledgeBaseId': '4NQZSAPNPH',
 'startedAt': datetime.datetime(2024, 5, 22, 1, 24, 24, 636367, tzinfo=tzlocal()),
 'statistics': {'numberOfDocumentsDeleted': 0,
                'numberOfDocumentsFailed': 0,
                'numberOfDocumentsScanned': 5,
                'numberOfMetadataDocumentsModified': 0,
                'numberOfMetadataDocumentsScanned': 0,
                'numberOfModifiedDocumentsIndexed': 0,
                'numberOfNewDocumentsIndexed': 5},
 'status': 'COMPLETE',
 'updatedAt': datetime.datetime(2024, 5, 22, 1, 24, 33, 838361, tzinfo=tzlocal())}


In [17]:
kb_id = kb["knowledgeBaseId"]
%store kb_id
pp.pprint(kb_id)

Stored 'kb_id' (str)
'4NQZSAPNPH'


## Test the knowledge base
### Using RetrieveAndGenerate API
Behind the scenes, RetrieveAndGenerate API converts queries into embeddings, searches the knowledge base, and then augments the foundation model prompt with the search results as context information and returns the FM-generated response to the question. For multi-turn conversations, Knowledge Bases manage short-term memory of the conversation to provide more contextual results.

The output of the RetrieveAndGenerate API includes the generated response, source attribution as well as the retrieved text chunks.

In [19]:
# try out KB using RetrieveAndGenerate API
bedrock_agent_runtime_client = boto3.client("bedrock-agent-runtime", region_name=region_name)
model_id = "anthropic.claude-3-sonnet-20240229-v1:0" # try with both claude instant as well as claude-v2. for claude v2 - "anthropic.claude-v2"
model_arn = f'arn:aws:bedrock:{region_name}::foundation-model/{model_id}'

In [20]:
from IPython.display import Markdown, display

query = "How do I add a new pet using the petstore api? Can you generate a test code in python?"
response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        'text': query
    },
    retrieveAndGenerateConfiguration={
        'type': 'KNOWLEDGE_BASE',
        'knowledgeBaseConfiguration': {
            'knowledgeBaseId': kb_id,
            'modelArn': model_arn
        }
    },
)

generated_text = response['output']['text']

display(Markdown(generated_text))

To add a new pet using the petstore API, you need to send a POST request to the /pets endpoint with a request body containing a NewPet object. The NewPet object should have the name property (required, string) and optionally the tag property (string).

Here's an example Python code to create a new pet using the requests library: import requests

url = "https://petstore.swagger.io/v2/pets"

new_pet = {
    "name": "Buddy",
    "tag": "dog"
}

response = requests.post(url, json=new_pet)

if response.status_code == 200:
    print("New pet created:", response.json())
else:
    print("Error creating pet:", response.status_code, response.text)

In [21]:
import requests
url = "http://petstore.swagger.io/v2/pet"

payload = { "name": "Buddy", "photoUrls": [ "http://pet-images.com/buddy.jpg" ], "category": { "id": 1, "name": "Dogs" }, "status": "available" } 
headers = { 'Content-Type': 'application/json' }

response = requests.request("POST", url, headers=headers, json=payload)

print(response.text)

<?xml version="1.0" encoding="UTF-8" standalone="yes"?><apiResponse><type>unknown</type></apiResponse>


## Convert Knowledgebase to Agent
Once the needed IAM role is created, we can use the bedrock agent client to create a new agent. To do so we use the create_agent function. It requires an agent name, underline foundation model and instruction. You can also provide an agent description. Note that the agent created is not yet prepared. We will focus on preparing the agent and then using it to invoke actions and use other APIs

In [21]:
# agent configuration
kb_agent_name = f'swagger-agent-{suffix}'

agent_instruction = """
You are an agent designed to support users with coding questions related to Swagger APIs. You have access to the Swagger
documentation in a Knowledge Base and you can answer questions from this document or use document as referecne to generate
code or flow diagrams. Only answer questions based on the documentation and reply with "There is no information about your 
question in the Documentation at the moment, sorry! Do you want to ask another question?"  If the answer to the question 
is not available in the documentation
"""

In [22]:
# Create Agent
response = bedrock_agent_client.create_agent(
    agentName=kb_agent_name,
    agentResourceRoleArn=roleArn,
    description="This Agent supports Swagger API Developers.",
    idleSessionTTLInSeconds=1800,
    foundationModel=model_id,
    instruction=agent_instruction,
)

In [23]:
agent_id = response['agent']['agentId']
agent_id

'PEJTE1OGIE'

### Associate agent to the Knowledge Base
Next, we need to associate the agent created with the Knowledge Base for the Bedrock documentation

In [24]:
agent_kb_description = bedrock_agent_client.associate_agent_knowledge_base(
    agentId=agent_id,
    agentVersion='DRAFT',
    description=f'Use the information in the {kb_name} knowledge base to provide accurate responses to the questions about Amazon Bedrock.',
    knowledgeBaseId=kb_id 
)

### Prepare Agent
Let's create a DRAFT version of the agent that can be used for internal testing.

In [25]:
agent_prepare = bedrock_agent_client.prepare_agent(agentId=agent_id)
agent_prepare

{'ResponseMetadata': {'RequestId': 'd2114e36-5749-42b9-bd0b-c75510081432',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Wed, 22 May 2024 01:29:02 GMT',
   'content-type': 'application/json',
   'content-length': '119',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'd2114e36-5749-42b9-bd0b-c75510081432',
   'x-amz-apigw-id': 'YJiO0HdBPHcEATQ=',
   'x-amzn-trace-id': 'Root=1-664d4a5e-451a513e301026db307a1d13'},
  'RetryAttempts': 0},
 'agentId': 'PEJTE1OGIE',
 'agentStatus': 'PREPARING',
 'agentVersion': 'DRAFT',
 'preparedAt': datetime.datetime(2024, 5, 22, 1, 29, 2, 567393, tzinfo=tzlocal())}

In [26]:
# Pause to make sure agent is prepared
time.sleep(30)
agent_alias_name = "swagger-docs-alias"

agent_alias = bedrock_agent_client.create_agent_alias(
    agentId=agent_id,
    agentAliasName=agent_alias_name
)
# Pause to make sure agent alias is ready
time.sleep(60)

Now that we've created the agent, let's use the bedrock-agent-runtime client to invoke this agent and get the information from the Knowledge base

## Test the Agent
Now that we've created the agent, let's use the bedrock-agent-runtime client to invoke this agent and perform some tasks. You can invoke your agent with the invoke_agent API

In [27]:
def invokeAgent(query, session_id, enable_trace=False, session_state=dict()):
    end_session:bool = False
    
    # invoke the agent API
    agentResponse = bedrock_agent_runtime_client.invoke_agent(
        inputText=query,
        agentId=agent_id,
        agentAliasId=agent_alias_id, 
        sessionId=session_id,
        enableTrace=enable_trace, 
        endSession= end_session,
        sessionState=session_state
    )
    
    if enable_trace:
        pp.pprint(agentResponse)
        pp.pprint("----------------------\n")
    
    event_stream = agentResponse['completion']
    try:
        for event in event_stream:        
            if 'chunk' in event:
                data = event['chunk']['bytes']
                if enable_trace:
                    pp.pprint(f"Final answer ->\n{data.decode('utf8')}")
                    pp.pprint("----------------------\n")
                agent_answer = data.decode('utf8')
                end_event_received = True
                return agent_answer
                # End event indicates that the request finished successfully
            elif 'trace' in event:
                if enable_trace:
                    pp.pprint(json.dumps(event['trace'], indent=2))
                    pp.pprint("----------------------\n")
            else:
                raise Exception("unexpected event.", event)
    except Exception as e:
        raise Exception("unexpected event.", e)

In [30]:
%%time
import uuid
agent_alias_id = agent_alias['agentAlias']['agentAliasId']
session_id:str = str(uuid.uuid1())
query = "How do I add a new pet using the petstore api?"
response = invokeAgent(query, session_id, enable_trace=False)
display(Markdown(response))

To create a new pet using the petstore API, you need to send a POST request to the /pets endpoint with a request body containing a NewPet object. The NewPet object should have the name property (required, string) and optionally the tag property (string). The response will be the newly created Pet object, which includes the id property in addition to the name and tag properties provided in the request.

CPU times: user 15.6 ms, sys: 0 ns, total: 15.6 ms
Wall time: 9.07 s
